In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp
import sys
import os

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

from registration_utilities import *

from skimage.measure import marching_cubes, correct_mesh_orientation

from vis3d_utilities import *
from itertools import izip

%matplotlib inline

Setting environment for Local Macbook Pro


In [2]:
stack_moving = 'MD594'
stack_fixed = 'MD589'

In [3]:
volume_fixed = bp.unpack_ndarray_file(DataManager.get_annotation_volume_filepath(stack=stack_fixed, downscale=32))
volume_moving = bp.unpack_ndarray_file(DataManager.get_annotation_volume_filepath(stack=stack_moving, downscale=32))
print volume_fixed.shape
print volume_moving.shape

(348, 421, 377)
(326, 515, 368)


In [6]:
# atlasAlignParams_dir = create_if_not_exists(atlasAlignParams_rootdir + '/%(stack_moving)s_to_%(stack_fixed)s' % \
#                      {'stack_moving': stack_moving, 'stack_fixed': stack_fixed})

In [7]:
# volume_fixed = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_annotationVolume.bp' % {'stack': stack_fixed}))
# volume_moving = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_annotationVolume.bp' % {'stack': stack_moving}))
# print volume_fixed.shape
# print volume_moving.shape

In [5]:
# with open(atlasAlignParams_dir + '/%(stack_moving)s_down32_annotationVolume_to_%(stack_fixed)s_down32_annotationVolume_parameters.txt' % \
#           {'stack_moving': stack_moving, 'stack_fixed': stack_fixed}, 'r') as f:
    
#     lines = f.readlines()
    
#     global_params = one_liner_to_arr(lines[0], float)
#     centroid_m = one_liner_to_arr(lines[1], float)
#     xdim_m, ydim_m, zdim_m  = one_liner_to_arr(lines[2], int)
#     centroid_f = one_liner_to_arr(lines[3], float)
#     xdim_f, ydim_f, zdim_f  = one_liner_to_arr(lines[4], int)

In [4]:
label_to_name_m, name_to_label_m = DataManager.load_volume_label_to_name(stack_moving)
label_to_name_f, name_to_label_f = DataManager.load_volume_label_to_name(stack_fixed)

In [5]:
colors = {name_u: np.random.randint(0,255,3)/255.
          for name_u in set([convert_name_to_unsided(name_s) for name_s in name_to_label_m.keys()])}

In [8]:
# Save meshes of moving volume aligned to fixed

In [11]:
moving_volume_alignedTo_fixed_volume_fn = os.path.join('/home/yuncong/CSHL_atlasAlignParams_atlas_v2/', stack_moving+'_to_'+stack_fixed, 
             "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume.bp" %\
             {'stack_moving': stack_moving, 'stack_fixed': stack_fixed})

In [12]:
volume_m_alignedTo_f = bp.unpack_ndarray_file(moving_volume_alignedTo_fixed_volume_fn)

In [13]:
vol_f_xmin, vol_f_xmax, vol_f_ymin, vol_f_ymax, vol_f_zmin, vol_f_zmax = DataManager.load_volume_bbox(stack_fixed, type='annotation', downscale=32)
print vol_f_xmin, vol_f_xmax, vol_f_ymin, vol_f_ymax, vol_f_zmin, vol_f_zmax

622 1042 155 502 127 503


In [14]:
m_alignedTo_f_meshes_dir = create_if_not_exists(os.path.join('/home/yuncong/CSHL_meshes_v2/', stack_moving+'_to_'+stack_fixed))

In [12]:
for name_s, label in name_to_label_m.iteritems():
    
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(volume_m_alignedTo_f == label)
    vol = (volume_m_alignedTo_f[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == label).astype(np.float)

    t = time.time()
    polydata = volume_to_polydata(vol, origin=(vol_f_xmin+xmin, vol_f_ymin+ymin, vol_f_zmin+zmin), 
                                  num_simplify_iter=3, smooth=True)
    sys.stderr.write('time: %.2f\n' % (time.time() - t) )

    polydata_fn = os.path.join(m_alignedTo_f_meshes_dir, 
                "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                  {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': name_s})
    
    save_mesh_stl(polydata, polydata_fn)

marching cube: 0.04 seconds
compute surface area: 0.00 seconds
fill point array: 0.02 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.02 seconds
simplify 0 @ 12931: 0.14 seconds
simplify 1 @ 2587: 0.11 seconds
simplify 2 @ 519: 0.02 seconds
time: 0.33
marching cube: 0.16 seconds
compute surface area: 0.01 seconds
fill point array: 0.04 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.04 seconds
simplify 0 @ 32510: 0.30 seconds
simplify 1 @ 6503: 0.27 seconds
simplify 2 @ 1302: 0.06 seconds
time: 0.84
marching cube: 0.04 seconds
compute surface area: 0.00 seconds
fill point array: 0.02 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.02 seconds
simplify 0 @ 16823: 0.15 seconds


KeyboardInterrupt: 

In [ ]:
# Load moving volume aligned to fixed

In [118]:
moving_volume_alignedTo_fixed_volume_polydata_list = {}

for name_s, label in name_to_label_m.iteritems():
    
    fn = os.path.join(m_alignedTo_f_meshes_dir, 
                      "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                      {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': name_s})
    
    moving_volume_alignedTo_fixed_volume_polydata_list[name_s] = load_mesh_stl(fn, return_polydata_only=True)

In [119]:
# Load moving volume

In [120]:
moving_volume_polydata_list = {}

for name_s, label in name_to_label_m.iteritems():
    
    fn = os.path.join(MESH_ROOTDIR, stack_moving, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack_moving, 'name': name_s})    
    moving_volume_polydata_list[name_s] = load_mesh_stl(fn, return_polydata_only=True)

In [121]:
# Load fixed volume

In [122]:
fixed_volume_polydata_list = {}

for name_s, label in name_to_label_f.iteritems():
    
    fn = os.path.join(MESH_ROOTDIR, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack_fixed, 'name': name_s})
    fixed_volume_polydata_list[name_s] =load_mesh_stl(fn, return_polydata_only=True)

In [123]:
# Show each structure in one color

volume_m_alignedTo_f_mesh_actors = [actor_mesh(polydata, colors[convert_name_to_unsided(name_s)], wireframe=True, opacity=1.) 
               for name_s, polydata in moving_volume_alignedTo_fixed_volume_polydata_list.iteritems()]

volume_f_mesh_actors = [actor_mesh(polydata, colors[convert_name_to_unsided(name_s)], wireframe=True, opacity=1.) 
               for name_s, polydata in fixed_volume_polydata_list.iteritems()]

launch_vtk(volume_m_alignedTo_f_mesh_actors + volume_f_mesh_actors)

In [124]:
# Show each brain in one color

volume_m_alignedTo_f_mesh_actors = [actor_mesh(polydata, (0,1,1), wireframe=False, opacity=.5) 
               for name_s, polydata in moving_volume_alignedTo_fixed_volume_polydata_list.iteritems()]

volume_f_mesh_actors = [actor_mesh(polydata, (0,0,1), wireframe=False, opacity=.5) 
               for name_s, polydata in fixed_volume_polydata_list.iteritems()]

volume_m_mesh_actors = [actor_mesh(polydata, (0,1,1), wireframe=False, opacity=.5) 
               for name_s, polydata in moving_volume_polydata_list.iteritems()]

launch_vtk(volume_m_alignedTo_f_mesh_actors + volume_f_mesh_actors + volume_m_mesh_actors)

In [71]:
launch_vtk(volume_m_alignedTo_f_mesh_actors + volume_f_mesh_actors)

In [66]:
launch_vtk(volume_f_mesh_actors + volume_m_mesh_actors, init_angle='30')

In [109]:
all_names = set([convert_name_to_unsided(name_s) for name_s in name_to_label_f.keys()])

In [110]:
centroid_allLandmarks = defaultdict(list)
polydata_list_allLandmarks = {}
average_polydata_allLandmarks = {}
average_volume_allLandmarks = {}
average_volume_origin_allLandmarks = {}

atlas_space_structure_volumes = {}
atlas_space_structure_meshes = {}

In [115]:
for name_u in all_names:

    print name_u
    
    vertices_list = []
    faces_list = []
    
    if name_u in singular_structures:
        
        fn = os.path.join(MESH_ROOTDIR, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % \
                              {'stack': stack_fixed, 'name': name_u})

        vertices, faces = load_mesh_stl(fn)

        vertices_mirrored = vertices.copy()
        zmean = vertices[:,2].mean(axis=0)
        vertices_mirrored[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        

        vertices_list.append(vertices)
        faces_list.append(faces)

        vertices_list.append(vertices_mirrored)
        faces_list.append(faces)
        
        centroid = vertices.mean(axis=0)
        centroid_allLandmarks[name_u].append(centroid)

#         centroid_anchor = centroid.copy()
        
    
    else:
    
        lname = convert_to_left_name(name_u)
        rname = convert_to_right_name(name_u)

        if lname in name_to_label_f:

            ############## Left #############

            fn = os.path.join(MESH_ROOTDIR, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % \
                              {'stack': stack_fixed, 'name': lname})

            vertices, faces = load_mesh_stl(fn)

            zmean = vertices[:,2].mean(axis=0)
            vertices[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin

            vertices_list.append(vertices)
            faces_list.append(faces)

            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[lname].append(centroid)

#             centroid_anchor = centroid.copy()


        if rname in name_to_label_f:

            ################# Right ############## 
            # Right shape is used as the template
            #####################################

            fn = os.path.join(MESH_ROOTDIR, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % \
                              {'stack': stack_fixed, 'name': rname})

            vertices, faces = load_mesh_stl(fn)

            vertices_list.append(vertices)
            faces_list.append(faces)

            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[rname].append(centroid)


        if lname in name_to_label_m:

            fn = os.path.join(m_alignedTo_f_meshes_dir, 
                          "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': lname})

            vertices, faces = load_mesh_stl(fn)

            zmean = vertices[:,2].mean(axis=0)
            vertices[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin

            vertices_list.append(vertices)
            faces_list.append(faces)


            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[lname].append(centroid)

        if rname in name_to_label_m:

            fn = os.path.join(m_alignedTo_f_meshes_dir, 
                          "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': rname})
            vertices, faces = load_mesh_stl(fn)

            vertices_list.append(vertices)
            faces_list.append(faces)

            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[rname].append(centroid)
        
        
    ##############################
    
    vertices_aligned_list = [] # all meshes in this list have vertices mean at (0,0)

    for i, vertices in enumerate(vertices_list):

        if i == 0:
            vertices1 = vertices.copy()
            vertices_aligned_list.append(vertices1 - vertices1.mean(axis=0))
        else:            
            t = time.time()                    
            vertices_alignedTo1 = icp(fixed_pts=vertices1, moving_pts=vertices, num_iter=100, rotation_only=True)
            sys.stderr.write('icp: %.2f\n' % (time.time() - t))

            vertices_aligned_list.append(vertices_alignedTo1 - vertices_alignedTo1.mean(axis=0))

    ################################
    
    polydata_list = [mesh_to_polydata(vs, fs) for vs, fs in zip(vertices_aligned_list, faces_list)]
    
    polydata_list_allLandmarks[name_u] = polydata_list
        
    average_volume, average_volume_origin, average_polydata = average_shape(polydata_list, num_simplify_iter=3, smooth=True)
    # these all have vertices mean at (0,0,0)
    
    average_polydata_allLandmarks[name_u] = average_polydata
    average_volume_allLandmarks[name_u] = average_volume
    average_volume_origin_allLandmarks[name_u] = average_volume_origin
    
#     instance_actors = [actor_mesh(polydata, np.random.randint(0,255,3)/255., opacity=.5, wireframe=True) 
#                    for polydata in polydata_list_allLandmarks[name_u]]

#     average_mesh_actor = actor_mesh(average_polydata_allLandmarks[name_u], (1,1,1), opacity=.5, wireframe=True)
#     launch_vtk(instance_actors + [average_mesh_actor])
    
#     average_volume_actor = actor_volume(average_volume.astype(np.float32), 
#                                         what='probability', origin=average_volume_origin)
#     launch_vtk(instance_actors + [average_volume_actor])

Tz
VCA
7n
DC
5N
3N
Pn
10N
VLL
7N
Amb
12N
RMC
Sp5O
Sp5I
Sp5C
VCP
AP
PBG
LRt
IC
LC
4N
SNC
6N
SNR
SC
RtTg


In [ ]:
# Take pics of different instances of 7N separately

for i, polydata in enumerate(polydata_list_allLandmarks['7N']):
    instance_actors = actor_mesh(polydata, [(1,0,0),(0,1,0),(0,0,1),(1,1,0)][i], opacity=1, wireframe=False)
    launch_vtk([instance_actors], snapshot_fn='/home/yuncong/7N_%d.png' % i, interactive=True, init_angle='coronal')

In [ ]:
# Take pic of different instances of 7N shown aligned together

instance_actors = [actor_mesh(polydata, [(1,0,0),(0,1,0),(0,0,1),(1,1,0)][i], opacity=.5, wireframe=True) 
                   for i, polydata in enumerate(polydata_list_allLandmarks['7N'])]
launch_vtk(instance_actors, snapshot_fn='/home/yuncong/7N_aligned.png', interactive=False, init_angle='coronal')

In [ ]:
# Take pic of merged probabilistic volume of 7N

average_volume_actor = actor_volume(average_volume_allLandmarks['7N'].astype(np.float32), 
                                        what='probability', origin=average_volume_origin_allLandmarks['7N'])
launch_vtk([average_volume_actor], snapshot_fn='/home/yuncong/7N_prob.png', interactive=False, init_angle='coronal')

In [ ]:
# Take pic of merged probabilistic volume of 7N, thresholded at .5

actor = actor_mesh(average_polydata_allLandmarks['7N'], [1,1,1], opacity=.5, wireframe=True)
launch_vtk([actor], snapshot_fn='/home/yuncong/7N_thresholded.png', interactive=False, init_angle='coronal')

In [27]:
canonical_locations, canonical_centroid, canonical_normal = average_location(centroid_allLandmarks)
# canonical locations are with respect to canonical centroid

[ 0.03550233 -0.09604324  0.99474382] @ [ 796.99383545  373.38763428  308.71014404]


In [94]:
print 'deviation angle:', np.rad2deg(np.arccos(canonical_normal[2]))

deviation angle: 5.87709878236


In [ ]:
for name_u in all_names:
    
    if name_u in singular_structures:
        canonical_structure_centroid = canonical_locations[name_u]

        average_volume = average_volume_allLandmarks[name_u]
        average_volume_origin = average_volume_origin_allLandmarks[name_u]
        average_polydata = average_polydata_allLandmarks[name_u]
    
        atlas_space_structure_volumes[name_u] = (average_volume, 
                                            canonical_structure_centroid + average_volume_origin)
        # new origin in the space
        
        vs, fs = polydata_to_mesh(average_polydata)
        atlas_space_structure_meshes[name_u] = mesh_to_polydata(vs + canonical_structure_centroid, fs)
        
    else:
    
        lname = convert_to_left_name(name_u)
        rname = convert_to_right_name(name_u)

        canonical_structure_centroid_r = canonical_locations[rname]
        canonical_structure_centroid_l = canonical_locations[lname]

        average_volume = average_volume_allLandmarks[name_u]
        average_volume_origin = average_volume_origin_allLandmarks[name_u]
        average_polydata = average_polydata_allLandmarks[name_u]

        
        # Restore left (non-template) structures, 
        # Note that mirror_volume() assumes flipping the shape
        # with respect to the original vertical plane (NOT canonical midplane !!!)
        atlas_space_structure_volumes[rname] = (average_volume, 
                                            canonical_structure_centroid_r + average_volume_origin)
        atlas_space_structure_volumes[lname] = mirror_volume(average_volume, 
                                             canonical_structure_centroid_l + average_volume_origin*(1,1,-1))

        vs, fs = polydata_to_mesh(average_polydata)
        atlas_space_structure_meshes[rname] = mesh_to_polydata(vs + canonical_structure_centroid_r, fs)
        atlas_space_structure_meshes[lname] = mesh_to_polydata(vs*(1,1,-1) + canonical_structure_centroid_l, fs)

In [107]:
volume_m.shape

(326, 515, 368)

In [ ]:
# Create an atlas proba. volume with all structures

atlas_prob_volume = np.zeros_like(volume_fixed)

for name_u in all_names:
    
    if name_u in singular_structures:

        volume, origin = atlas_space_structure_volumes[name_u]
        
        ydim, xdim, zdim = volume.shape
        
        # Convert canonical locations to actual location
        # Being lazy here: the z axis should be orthogonal to the canonical normal, but here I assume z axis is the original
        y, x, z = origin + canonical_centroid
        
        atlas_prob_volume[y:y+ydim, x:x+xdim, z:z+zdim] = volume
        
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

    else:
    
        lname = convert_to_left_name(name_u)
        volume, origin = atlas_space_structure_volumes[lname]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

        rname = convert_to_right_name(name_u)
        volume, origin = atlas_space_structure_volumes[rname]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

In [29]:
all_atlas_structure_volume_actors = []

for name_u in all_names:
    
    if name_u in singular_structures:

        volume, origin = atlas_space_structure_volumes[name_u]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

    else:
    
        lname = convert_to_left_name(name_u)
        volume, origin = atlas_space_structure_volumes[lname]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

        rname = convert_to_right_name(name_u)
        volume, origin = atlas_space_structure_volumes[rname]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

In [30]:
launch_vtk(all_atlas_structure_volume_actors, init_angle='sagittal', background_color=(0,0,0))

In [31]:
create_if_not_exists(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_mesh'))

'/home/yuncong/CSHL_meshes/atlas/structure_mesh'

In [32]:
for name_s, p in atlas_space_structure_meshes.iteritems():
    save_mesh_stl(p, os.path.join(MESH_ROOTDIR, 'atlas', 'structure_mesh', 'mesh_%s.stl'%name_s))

In [33]:
create_if_not_exists(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume'))

'/home/yuncong/CSHL_meshes/atlas/structure_prob_volume'

In [34]:
for name_s, (volume, origin) in atlas_space_structure_volumes.iteritems():
    bp.pack_ndarray_file(volume, os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_%s.bp'%name_s))
    np.savetxt(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_origin_%s.txt'%name_s), origin)

In [35]:
def write_dict_to_txt(d, fn):
    with open(fn, 'w') as f:
        for k, vals in d.iteritems():
            f.write(k + ' ' +  (' '.join(['%f']*len(vals))) % tuple(vals) + '\n')

In [36]:
write_dict_to_txt(canonical_locations, os.path.join(MESH_ROOTDIR, 'atlas', 'canonical_locations.txt'))
np.savetxt(os.path.join(MESH_ROOTDIR, 'atlas', 'canonical_centroid.txt'), canonical_centroid)
np.savetxt(os.path.join(MESH_ROOTDIR, 'atlas', 'canonical_normal.txt'), canonical_normal)

In [21]:
def read_dict_from_txt(fn):
    d = {}
    with open(fn, 'r') as f:
        for line in f.readlines():
            items = line.split()
            d[items[0]] = np.array(map(np.float, items[1:]))
    return d

In [22]:
# Load canonical info

canonical_locations = read_dict_from_txt(os.path.join(MESH_ROOTDIR, 'atlas', 'canonical_locations.txt'))
canonical_centroid = np.loadtxt(os.path.join(MESH_ROOTDIR, 'atlas', 'canonical_centroid.txt'))
canonical_normal = np.loadtxt(os.path.join(MESH_ROOTDIR, 'atlas', 'canonical_normal.txt'))

In [23]:
# Load atlas meshes

atlas_space_structure_meshes = {}

for name_u in all_names:
# for name_u in ['Tz']:
    
    print name_u
    
    if name_u in singular_structures:
        polydata = load_mesh_stl(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_mesh', 'mesh_%s.stl'%name_u),
                                                            return_polydata_only=True)
#         atlas_space_structure_meshes[name_u] = poisson_reconstruct_meshlab(polydata)
        atlas_space_structure_meshes[name_u] = polydata
        
    else:
    
        lname = convert_to_left_name(name_u)
        rname = convert_to_right_name(name_u)

        polydata_l = load_mesh_stl(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_mesh', 'mesh_%s.stl'%lname),
                                                           return_polydata_only=True)
#         atlas_space_structure_meshes[lname] = poisson_reconstruct_meshlab(polydata_l)
        atlas_space_structure_meshes[lname] = polydata_l
            
        polydata_r = load_mesh_stl(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_mesh', 'mesh_%s.stl'%rname),
                                                           return_polydata_only=True)
#         atlas_space_structure_meshes[rname] = poisson_reconstruct_meshlab(polydata_r)
        atlas_space_structure_meshes[rname] = polydata_r

Tz
VCA
7n
DC
5N
3N
Pn
10N
VLL
7N
Amb
12N
RMC
Sp5O
Sp5I
Sp5C
VCP
AP
PBG
LRt
IC
LC
4N
SNC
6N
SNR
SC
RtTg


In [24]:
# Create atlas mesh actors

all_atlas_structure_mesh_actors = []

for name_u in all_names:
# for name_u in ['7N', '5N', '12N']:

    
    if name_u in singular_structures:
        
        structure_mesh_actor = actor_mesh(atlas_space_structure_meshes[name_u], color=colors[name_u], opacity=.5)
        all_atlas_structure_mesh_actors.append(structure_mesh_actor)
        
    else:
    
        lname = convert_to_left_name(name_u)
        structure_mesh_actor = actor_mesh(atlas_space_structure_meshes[lname], color=colors[name_u], opacity=.5)
        all_atlas_structure_mesh_actors.append(structure_mesh_actor)

        rname = convert_to_right_name(name_u)
        structure_mesh_actor = actor_mesh(atlas_space_structure_meshes[rname], color=colors[name_u], opacity=.5)
        all_atlas_structure_mesh_actors.append(structure_mesh_actor)

In [25]:
# Load structure prob volumes

atlas_space_structure_volumes = {}

for name_u in all_names:
    
    if name_u in singular_structures:
        volume = bp.unpack_ndarray_file(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_%s.bp'%name_u))
        origin = np.loadtxt(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_origin_%s.txt'%name_u))

        atlas_space_structure_volumes[name_u] = (volume, origin)
        
    
    else:
        lname = convert_to_left_name(name_u)
            
        volume = bp.unpack_ndarray_file(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_%s.bp'%lname))
        origin = np.loadtxt(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_origin_%s.txt'%lname))

        atlas_space_structure_volumes[lname] = (volume, origin)
        
        rname = convert_to_right_name(name_u)
        
        volume = bp.unpack_ndarray_file(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_%s.bp'%rname))
        origin = np.loadtxt(os.path.join(MESH_ROOTDIR, 'atlas', 'structure_prob_volume', 'prob_volume_origin_%s.txt'%rname))

        atlas_space_structure_volumes[rname] = (volume, origin)

In [78]:
bbox = {}
for name_s, (vol, origin) in atlas_space_structure_volumes.iteritems():
    ydim, xdim, zdim = vol.shape
    bbox[name_s] = (origin, origin + (xdim, ydim, zdim))
    print name_s, bbox

LRt_R {'LRt_R': (array([ 78.31095847,  70.96773129,  47.59007675]), array([ 159.31095847,  120.96773129,  105.59007675]))}
Sp5C_L {'LRt_R': (array([ 78.31095847,  70.96773129,  47.59007675]), array([ 159.31095847,  120.96773129,  105.59007675])), 'Sp5C_L': (array([ 114.55138966,    1.17073308, -136.61091635]), array([ 240.55138966,  117.17073308,  -60.61091635]))}
Pn_L {'LRt_R': (array([ 78.31095847,  70.96773129,  47.59007675]), array([ 159.31095847,  120.96773129,  105.59007675])), 'Sp5C_L': (array([ 114.55138966,    1.17073308, -136.61091635]), array([ 240.55138966,  117.17073308,  -60.61091635])), 'Pn_L': (array([-141.26360351,   21.74822927,  -86.11736609]), array([-83.26360351,  95.74822927,  -8.11736609]))}
7N_R {'LRt_R': (array([ 78.31095847,  70.96773129,  47.59007675]), array([ 159.31095847,  120.96773129,  105.59007675])), '7N_R': (array([-15.19212124,  60.38365961,  45.74033752]), array([  43.80787876,  101.38365961,  111.74033752])), 'Sp5C_L': (array([ 114.55138966,    1.1

In [76]:

for name_s, (vol, origin) in atlas_space_structure_volumes.iteritems():
    
    

{'10N_L': (array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
  
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
  
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
  
         ..., 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0

In [26]:
# Create actors for structure prob volumes

all_atlas_structure_volume_actors = []

for name_s, (volume, origin) in atlas_space_structure_volumes.iteritems():
    
    name_u = convert_name_to_unsided(name_s)

    structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])
    all_atlas_structure_volume_actors.append(structure_volume_actor)

In [27]:
launch_vtk(all_atlas_structure_mesh_actors, init_angle='sagittal')

In [72]:
launch_vtk(all_atlas_structure_mesh_actors + all_atlas_structure_volume_actors, init_angle='sagittal')

In [73]:
fn = os.path.join(MESH_ROOTDIR, stack_fixed, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack_fixed})
outerContour_polydata = load_mesh_stl(fn, return_polydata_only=True)

outerContour_polydata = move_polydata(outerContour_polydata, -canonical_centroid)

outerContour_actor = actor_mesh(outerContour_polydata, (1,1,1), opacity=.15, wireframe=False)

In [40]:
launch_vtk(all_atlas_structure_mesh_actors + [outerContour_actor], init_angle='30')

In [32]:
launch_vtk(all_atlas_structure_mesh_actors + [outerContour_actor], init_angle='sagittal')

In [33]:
launch_vtk(all_atlas_structure_mesh_actors + [outerContour_actor], init_angle='horizontal_topDown')

In [31]:
launch_vtk(all_atlas_structure_mesh_actors + [outerContour_actor], init_angle='horizontal_topDown', animate=False,
          interactive=False, snapshot_fn='/home/yuncong/snapshot_horizontal.png', snapshot_magnification=10)

In [30]:
launch_vtk(all_atlas_structure_mesh_actors + [outerContour_actor], init_angle='sagittal', animate=False, 
           interactive=False, snapshot_fn='/home/yuncong/snapshot_sagittal.png', snapshot_magnification=10)

In [38]:
launch_vtk(all_atlas_structure_volume_actors + [outerContour_actor], init_angle='sagittal', animate=True)

In [74]:
launch_vtk(all_atlas_structure_volume_actors + [outerContour_actor], init_angle='sagittal')

In [75]:
launch_vtk(all_atlas_structure_volume_actors , init_angle='sagittal')

In [ ]:
# from collections import defaultdict
# centroid_allLandmarks = defaultdict(list)
# average_polydata_allLandmarks = {}
# polydata_list_allLandmarks = {}
# average_volume_allLandmarks = {}

# # for name_unsided in labels_unsided:
# # # for name_unsided in ['outerContour']:
# # # for name_unsided in ['Pn']:
    
# #     if name_unsided == 'outerContour' or name_unsided == 'sp5':
# #         continue
        
# #     names = labelMap_unsidedToSided[name_unsided]
# #     print name_unsided, names

# for name in names:
    
#     vertices_list = []
#     faces_list = []
    
#     for stack in ['MD589', 'MD594']:
# #     for stack in ['MD594']:

#         if len(names) == 2:
        
#             for name_sided in names:

#                 fn = "/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_gaussianSmoothed.stl" % {'stack':stack, 'label':name_sided}
               
#                 if os.path.exists(fn):
#                     vertices, faces = load_mesh_stl(fn)
#                 else:
#                     continue
            
#                 if '_L' in name_sided:
#                     zmean = vertices[:,2].mean(axis=0)
#                     vertices[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        

#                 if stack == 'MD594':
#                     vertices_alignedToScoreVolume = vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                     vertices = transform_points_inverse(global_params, pts_prime=vertices_alignedToScoreVolume, 
#                                                                        c_prime=test_centroid, c=atlas_centroid)
                    
#                 vertices_list.append(vertices)    
#                 faces_list.append(faces)
                
#                 centroid_prime = vertices.mean(axis=0)

#                 if stack == 'MD594':

#                     centroid_prime_alignedToScoreVolume = centroid_prime + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                     centroid_prime_alignedToAtlas = transform_points_inverse(global_params, pts_prime=[centroid_prime_alignedToScoreVolume], 
#                                                                        c_prime=test_centroid, c=atlas_centroid)[0]
#                     centroid_allLandmarks[name_sided].append(centroid_prime_alignedToAtlas)
#                 else:
#                     centroid_allLandmarks[name_sided].append(centroid_prime)
        
#         elif len(names) == 1:
            
#             name_sided = names[0]
            
#             fn = "/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_gaussianSmoothed.stl" % {'stack':stack, 'label':name_sided}
            
#             if os.path.exists(fn):
#                 vertices, faces = load_mesh_stl(fn)
#             else:
#                 continue

#             vertices_mirrored = vertices.copy()    
#             zmean = vertices[:,2].mean(axis=0)
#             vertices_mirrored[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        
            
#             t = time.time()
            
#             if name_unsided == 'sp5' or name_unsided == 'outerContour':
# #                 _, symmetric_poly = average_shape([mesh_to_polydata(vertices, faces), mesh_to_polydata(vertices_mirrored, faces)],
# #                                               num_simplify_iter=5, smooth=True)
#                 symmetric_poly = mesh_to_polydata(vertices, faces)
#             else:
#                 _, symmetric_poly = average_shape([mesh_to_polydata(vertices, faces), mesh_to_polydata(vertices_mirrored, faces)],
#                                               num_simplify_iter=3, smooth=True)
                
#             # must simplify so the ICP later can be fast
#             # must also smooth, because otherwise the number of points gets stuck
            
#             sys.stderr.write('average shape: %.2f\n' % (time.time() - t))
#             # cost mostly comes from :
#             # - marching cube
#             # - first two rounds of decimation and smoothing

#             symmetric_vertices, symmetric_faces = polydata_to_mesh(symmetric_poly)
         
#             if stack == 'MD594':
#                 vertices_alignedToScoreVolume = symmetric_vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                 symmetric_vertices = transform_points_inverse(global_params, pts_prime=vertices_alignedToScoreVolume, 
#                                                                    c_prime=test_centroid, c=atlas_centroid)
        
#             vertices_list.append(symmetric_vertices)
#             faces_list.append(symmetric_faces)
            
#             centroid_prime = symmetric_vertices.mean(axis=0)

#             if stack == 'MD594':

#                 centroid_prime_alignedToScoreVolume = centroid_prime + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                 centroid_prime_alignedToAtlas = transform_points_inverse(global_params, pts_prime=[centroid_prime_alignedToScoreVolume], 
#                                                                    c_prime=test_centroid, c=atlas_centroid)[0]
#                 centroid_allLandmarks[name_sided].append(centroid_prime_alignedToAtlas)
#             else:
#                 centroid_allLandmarks[name_sided].append(centroid_prime)
    
#     ###### Align meshes ######
    
#     vertices_aligned_list = []

#     for i, vertices in enumerate(vertices_list):

# #         if i == 0:
# #             vertices1 = vertices.copy()
# #             vol_fixed, origin_fixed, _ = polydata_to_volume(mesh_to_polydata(vertices, faces_list[0]))
# #             vertices_aligned_list.append(vertices1)
# #         else:            
# #             t = time.time()
# #             vol_moving, origin_moving, _ = polydata_to_volume(mesh_to_polydata(vertices, faces_list[i]))
# #             T, c_fixed, c_moving = optimize_rigid_lieGroup(vol_fixed, vol_moving)
# #             vertices_alignedTo1 = transform_points(T, vertices-origin_moving, c=c_moving, c_prime=c_fixed) + origin_fixed            
# #             sys.stderr.write('align: %.2f\n' % (time.time() - t))

# #             vertices_aligned_list.append(vertices_alignedTo1)
                
#         if i == 0:
#             vertices1 = vertices.copy()
#             vertices_aligned_list.append(vertices1)
#         else:            
#             t = time.time()                    
#             vertices_alignedTo1 = icp(fixed_pts=vertices1, moving_pts=vertices, num_iter=100)
#             sys.stderr.write('icp: %.2f\n' % (time.time() - t))

#             vertices_aligned_list.append(vertices_alignedTo1)
    
#     polydata_list = [mesh_to_polydata(vs, fs) for vs, fs in zip(vertices_aligned_list, faces_list)]
            
#     # if landmark has only one instance, add its mirrored version
#     if len(names) == 1:
#         for vs, fs in zip(vertices_aligned_list, faces_list):
#             zmean = vs[:,2].mean(axis=0)
#             vs_mirrored = vs.copy()
#             vs_mirrored[:, 2] = - (vs[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        
#             polydata_list.append(mesh_to_polydata(vs_mirrored, fs))

#     polydata_list_allLandmarks[name_unsided] = polydata_list
    
# #     ######### Compute Average #########
#     t = time.time()

#     if name_unsided == 'outerContour' or name_unsided == 'sp5':
#         # since alignment is not ideal, just use MD589's outerContour for now
# #         average_volume, average_polydata = average_shape([polydata_list[0],polydata_list[2]], num_simplify_iter=5, smooth=True, concensus_percentage=0.9)
# #         average_volume, average_polydata = average_shape([polydata_list[0],polydata_list[0]], num_simplify_iter=5, smooth=False, concensus_percentage=0.9)
#         average_volume = None
#         average_polydata = polydata_list[2]
#     elif len(names) == 1:
#         average_volume, average_polydata = average_shape(polydata_list, num_simplify_iter=3, smooth=True, concensus_percentage=0.9)
#     else:
#         average_volume, average_polydata = average_shape(polydata_list, num_simplify_iter=3, smooth=True)
    
#     sys.stderr.write('average shape: %.2f\n' % (time.time() - t))

#     average_polydata_allLandmarks[name_unsided] = average_polydata

#     average_volume_allLandmarks[name_unsided] = average_volume
    
# #     bp.pack_ndarray_file(average_volume, mesh_rootdir + "/%(name)s_average.bp" % {'stack': stack, 'name': name_unsided})
# #     save_mesh_stl(average_polydata, mesh_rootdir + "/%(name)s_average.stl" % {'stack': stack, 'name': name_unsided})

# centroid_allLandmarks.default_factory = None